<a href="https://colab.research.google.com/github/Yiting916/ML/blob/main/3-01.LAB_Automobile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 汽車價格預測

In [3]:
#載入訓練用資料
import pandas as pd
import numpy as np
data = pd.read_csv('/content/drive/MyDrive/YiTing/Learning/Python/ML/Automobile_Train.csv')
data.head()

,symboling,Normalized-losses,make,Fuel-type,aspiration,Num-of-doors,Body-style,Drive-wheels,Engine-location,Wheel-base,...,Engine-size,Fuel-system,bore,stroke,Compression-ratio,horsepower,Peak-rpm,City-mpg,Highway-mpg,price
0,1,NaN,porsche,gas,std,two,hatchback,rwd,front,98.4,...,203,mpfi,3.94,3.11,10.0,288.0,5750.0,17,28,NaN
1,3,256.0,volkswagen,gas,std,two,hatchback,fwd,front,94.5,...,109,mpfi,3.19,3.40,8.5,90.0,5500.0,24,29,9980.0
2,2,192.0,bmw,gas,std,two,sedan,rwd,front,101.2,...,108,mpfi,3.50,2.80,8.8,101.0,5800.0,23,29,16430.0
3,0,110.0,honda,gas,std,four,sedan,fwd,front,96.5,...,92,1bbl,2.91,3.41,9.2,76.0,6000.0,30,34,7295.0
4,2,137.0,honda,gas,std,two,hatchback,fwd,front,86.6,...,92,1bbl,2.91,3.41,9.2,76.0,6000.0,31,38,6855.0


In [4]:
#列出欄位名稱
data.columns

Index(['symboling', 'Normalized-losses', 'make', 'Fuel-type', 'aspiration',
       'Num-of-doors', 'Body-style', 'Drive-wheels', 'Engine-location',
       'Wheel-base', 'length', 'width', 'height', 'Curb-weight', 'Engine-type',
       'Num-of-cylinders', 'Engine-size', 'Fuel-system', 'bore', 'stroke',
       'Compression-ratio', 'horsepower', 'Peak-rpm', 'City-mpg',
       'Highway-mpg', 'price'],
      dtype='object')

In [5]:
# 查看資料訊息
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          150 non-null    int64  
 1   Normalized-losses  121 non-null    float64
 2   make               150 non-null    object 
 3   Fuel-type          150 non-null    object 
 4   aspiration         150 non-null    object 
 5   Num-of-doors       149 non-null    object 
 6   Body-style         150 non-null    object 
 7   Drive-wheels       150 non-null    object 
 8   Engine-location    150 non-null    object 
 9   Wheel-base         150 non-null    float64
 10  length             150 non-null    float64
 11  width              150 non-null    float64
 12  height             150 non-null    float64
 13  Curb-weight        150 non-null    int64  
 14  Engine-type        150 non-null    object 
 15  Num-of-cylinders   150 non-null    object 
 16  Engine-size        150 non

In [7]:
data.isnull().sum()

,0
symboling,0
Normalized-losses,29
make,0
Fuel-type,0
aspiration,0
Num-of-doors,1
Body-style,0
Drive-wheels,0
Engine-location,0
Wheel-base,0


# 補齊資料空值方法

資料空值補齊（Data Imputation）是資料分析中常見的處理缺失值技術。以下是常見的補齊方法：

## 1. 刪除法（Deletion）
- **單一刪除法（Listwise Deletion）**：若某筆資料的任一變數有缺失，則整筆資料會被刪除。適用於資料缺失少且不會造成顯著損失的情況。
- **變數刪除法（Pairwise Deletion）**：當計算某一變數之間的相關性時，僅刪除包含缺失值的部分資料，而不是整筆資料。

## 2. 均值/中位數/眾數插補（Mean/Median/Mode Imputation）-> 最簡單的無腦補法
- **均值插補**：對數值型資料，將缺失值用該變數的均值進行替代。適用於數據分佈較為對稱且缺失值較少的情況。
- **中位數插補**：對數值型資料，將缺失值用該變數的中位數進行替代。對於具有偏態分佈的資料，使用中位數比均值插補效果較好。
- **眾數插補**：對類別型資料，將缺失值用該變數的眾數（最常見的值）進行替代。

## 3. 線性回歸插補（Linear Regression Imputation）
- 根據其他變數（自變數）來預測缺失變數（因變數）的值，通過回歸分析將缺失的數值填補。例如，可以利用已有資料來訓練回歸模型，然後預測缺失值。

## 4. K近鄰插補（K-Nearest Neighbors Imputation, KNN）
- 根據缺失資料的相似度（距離度量），用最相似的K個觀測值的平均數或加權平均數來填補缺失值。此方法較為靈活，能夠考慮到資料之間的非線性關係。

## 5. 多重插補（Multiple Imputation）
- 通過多次插補生成多個缺失值填補結果，並考慮插補過程的不確定性。每次插補後，對模型進行分析，最終匯總結果。這種方法能夠更好地反映缺失值的變異性和不確定性。

## 6. 基於模型的插補（Model-based Imputation）
- 使用統計模型或機器學習方法（如隨機森林、貝葉斯模型等）進行插補。這些方法能夠根據資料的其他變數來估算缺失值，通常比簡單的均值插補效果更好。

## 7. 插補變數（Indicator Variable for Missingness）
- 當缺失值的存在本身具有某些含義時，可以創建一個指示變數（指示是否為缺失值），以便模型能夠理解缺失值的模式。

## 8. 前後觀測插補（Forward/Backward Fill）
- 在時間序列資料中，常用前一個觀測值（前向填補）或下一個觀測值（後向填補）來填補缺失值。這種方法適用於時間序列資料，能夠保持資料的時間順序。

## 9. 插值法（Interpolation）
- 針對時間序列資料，可以使用線性插值、樣條插值等方法來根據鄰近的數據點推測缺失值。


In [8]:
# 將 Normalized-losses 的空值補上中位數
# m = data['Normalized-losses'].median() # 先算出中位數
# data['Normalized-losses'] = data['Normalized-losses'].fillna(m) # 用 fillna 方法塞中位數
data['Normalized-losses'] = data['Normalized-losses'].fillna(data['Normalized-losses'].median())

In [10]:
# Num-of-doors : Label型資料
data['Num-of-doors'].value_counts()

,count
Num-of-doors,


In [11]:
data['Num-of-doors'] = data['Num-of-doors'].fillna('four')

In [12]:
# 補 bore, stroke, horsepower, Peak-rpm的空值
data['bore'] = data['bore'].fillna(data['bore'].median())
data['stroke'] = data['stroke'].fillna(data['stroke'].median())
data['horsepower'] = data['horsepower'].fillna(data['horsepower'].median())
data['Peak-rpm'] = data['Peak-rpm'].fillna(data['Peak-rpm'].median())

In [15]:
# 刪除 price為空值的資料列 -> 價格是要預測的目標, 不能補值, 所以直接刪除空的
data = data.dropna(subset=['price'])

In [16]:
# 再次檢查資料是否有空值
data.isnull().sum()

,0
symboling,0
Normalized-losses,0
make,0
Fuel-type,0
aspiration,0
Num-of-doors,0
Body-style,0
Drive-wheels,0
Engine-location,0
Wheel-base,0


# 資料編碼

# 訓練模型

# 測試與推理